# Modelos de ML

**Objetivo**: Criar modelos de ML para a projeção de todas as nossas séries.

**Metodologias**: 

- Regressão Linear
- Árvore de decisão
- Random Forest
- XGBoost
- LightGBM

## 0. Setup

In [1]:
%load_ext autotime

time: 172 µs (started: 2024-01-03 17:09:21 -03:00)


In [2]:
#---- Manipulação de dados:

import pandas as pd
import numpy as np

#---- Modelagem:

from hierarchicalforecast.utils import aggregate
from mlforecast import MLForecast
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

#---- Reconciliação

from hierarchicalforecast.methods import BottomUp, TopDown, ERM, OptimalCombination, MinTrace, MiddleOut
from hierarchicalforecast.core import HierarchicalReconciliation

#---- Visualização

import plotly.express as px

time: 1.33 s (started: 2024-01-03 17:09:21 -03:00)


## 1. Dados: vendas de roupas no varejo

In [3]:
dados = pd.read_csv('https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-hierarchical-forecasting/main/retail-usa-clothing.csv')

dados.head()

,date,state,item,quantity,region,country
0,1997-11-25,NewYork,mens_clothing,8,Mid-Alantic,USA
1,1997-11-26,NewYork,mens_clothing,9,Mid-Alantic,USA
2,1997-11-27,NewYork,mens_clothing,11,Mid-Alantic,USA
3,1997-11-28,NewYork,mens_clothing,11,Mid-Alantic,USA
4,1997-11-29,NewYork,mens_clothing,10,Mid-Alantic,USA


time: 1.14 s (started: 2024-01-03 17:09:22 -03:00)


In [4]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388024 entries, 0 to 388023
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   date      388024 non-null  object
 1   state     388024 non-null  object
 2   item      388024 non-null  object
 3   quantity  388024 non-null  int64 
 4   region    388024 non-null  object
 5   country   388024 non-null  object
dtypes: int64(1), object(5)
memory usage: 17.8+ MB
time: 54.8 ms (started: 2024-01-03 17:09:24 -03:00)


## 2. Modificação nos dados 

In [5]:
def clean_data_baseline(df: pd.DataFrame):

    #---- 1. Excluindo a variável de country:

    df = df\
        .drop(columns = 'country')

    #---- 2. Mudando o tipo da variável de date para datetime:

    df['date'] = pd.to_datetime(df['date'])

    #---- 3. Renomeando as variáveis de quantidade de vendas e data:
    # date -> ds
    # quantity -> y

    df = df\
        .rename(columns = {'date': 'ds', 
                           'quantity': 'y'})

    return df

time: 1.22 ms (started: 2024-01-03 17:09:24 -03:00)


In [6]:
df = clean_data_baseline(df = dados)

df.head()

,ds,state,item,y,region
0,1997-11-25,NewYork,mens_clothing,8,Mid-Alantic
1,1997-11-26,NewYork,mens_clothing,9,Mid-Alantic
2,1997-11-27,NewYork,mens_clothing,11,Mid-Alantic
3,1997-11-28,NewYork,mens_clothing,11,Mid-Alantic
4,1997-11-29,NewYork,mens_clothing,10,Mid-Alantic


time: 99.5 ms (started: 2024-01-03 17:09:24 -03:00)


In [7]:
def format_hierarchical_df(df: pd.DataFrame, cols_hierarchical: list):

    #---- 1. Cria uma lista de listas: [[col1], [col1, col2], ..., [col1, col2, coln]]

    hier_list = [cols_hierarchical[:i] for i in range(1, len(cols_hierarchical) + 1)]

    #---- 2. Aplica a função aggregate que formata os dados em que a lib hierarchical pede

    Y_df, S_df, tags = aggregate(df = df, spec = hier_list)

    return Y_df, S_df, tags

time: 933 µs (started: 2024-01-03 17:09:24 -03:00)


In [8]:
cols_hierarchical = ['region', 'state', 'item']

Y_df, S_df, tags = format_hierarchical_df(df = df, cols_hierarchical = cols_hierarchical)

time: 664 ms (started: 2024-01-03 17:09:24 -03:00)


In [9]:
display(Y_df.head())
display(Y_df.tail())

,ds,y
unique_id,,
EastNorthCentral,1997-11-25,507
EastNorthCentral,1997-11-26,504
EastNorthCentral,1997-11-27,510
EastNorthCentral,1997-11-28,507
EastNorthCentral,1997-11-29,513


,ds,y
unique_id,,
SouthCentral/Tennessee/womens_shoes,2009-07-24,31
SouthCentral/Tennessee/womens_shoes,2009-07-25,30
SouthCentral/Tennessee/womens_shoes,2009-07-26,31
SouthCentral/Tennessee/womens_shoes,2009-07-27,29
SouthCentral/Tennessee/womens_shoes,2009-07-28,30


time: 9.46 ms (started: 2024-01-03 17:09:24 -03:00)


- **Dados de treino: 25/11/1997 a 31/12/2008**
- **Dados de validação: 01/01/2009 a 28/07/2009**

In [10]:
def split_train_test(df: pd.DataFrame, dt_start_train: str):

    #---- 1. Dados de treino

    train = df.query(f'ds < "{dt_start_train}"')

    #---- 2. Dados de teste:
    
    valid = df.query(f'ds >= "{dt_start_train}"')

    return train, valid

time: 493 µs (started: 2024-01-03 17:09:24 -03:00)


In [11]:
Y_train_df, Y_valid_df = split_train_test(df = Y_df, dt_start_train = '2009-01-01')

time: 74.7 ms (started: 2024-01-03 17:09:24 -03:00)


In [12]:
display(Y_train_df.head())
display(Y_train_df.tail())

,ds,y
unique_id,,
EastNorthCentral,1997-11-25,507
EastNorthCentral,1997-11-26,504
EastNorthCentral,1997-11-27,510
EastNorthCentral,1997-11-28,507
EastNorthCentral,1997-11-29,513


,ds,y
unique_id,,
SouthCentral/Tennessee/womens_shoes,2008-12-27,31
SouthCentral/Tennessee/womens_shoes,2008-12-28,29
SouthCentral/Tennessee/womens_shoes,2008-12-29,28
SouthCentral/Tennessee/womens_shoes,2008-12-30,31
SouthCentral/Tennessee/womens_shoes,2008-12-31,31


time: 9.02 ms (started: 2024-01-03 17:09:24 -03:00)


## 3. Modelagem

### 3.1. Fit do modelo

In [13]:
#---- Modelos:

lin_reg = LinearRegression() # Regressão linear

dec_tree = DecisionTreeRegressor(random_state = 19, ) # Árvore de decisão

ran_forest = RandomForestRegressor(random_state = 19, n_estimators = 500) # Random Forest

lgbm = LGBMRegressor(random_state = 19, n_estimators = 500) # LightGBM

xgb = XGBRegressor(random_state = 19, n_estimators = 500) # XGBoost

models_list = [lin_reg, dec_tree, ran_forest, lgbm, xgb]

time: 2.16 ms (started: 2024-01-03 17:09:24 -03:00)


In [14]:
#---- Features de data:

from numba import njit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean

@njit
def rolling_mean_7(x):
    return rolling_mean(x, window_size = 7)

@njit
def rolling_mean_14(x):
    return rolling_mean(x, window_size = 14)

@njit
def rolling_mean_21(x):
    return rolling_mean(x, window_size = 21)

@njit
def rolling_mean_28(x):
    return rolling_mean(x, window_size = 28)

time: 7.88 ms (started: 2024-01-03 17:09:24 -03:00)


In [15]:
#---- Fit:

model = MLForecast(models = models_list,
                   freq = 'D',
                   num_threads = 6,
                   lags = [1, 7, 14, 21, 28, 30], 
                   date_features = ['dayofweek', 'month', 'year', 'quarter', 'day', 'week'],
                   lag_transforms = {
                       1: [expanding_mean],
                       7: [rolling_mean_7],
                       14: [rolling_mean_14],
                       21: [rolling_mean_21],
                       28: [rolling_mean_28],
                   }
           )

model.fit(Y_train_df.reset_index(), id_col = 'unique_id', time_col = 'ds', target_col = 'y', fitted = True)

KeyboardInterrupt: 

time: 23min 4s (started: 2024-01-03 17:09:24 -03:00)


In [ ]:
model.preprocess(Y_train_df.reset_index())

### 3.2. Predict para comparação entre os dados de validação

In [ ]:
n_horizon = Y_valid_df.ds.nunique() # Quantidade de dias para a projeção

Y_hat_df = model.predict(h = n_horizon)

In [ ]:
Y_hat_df.head()

In [ ]:
Y_fitted_df = model.forecast_fitted_values()

display(Y_fitted_df.head())
display(Y_fitted_df.tail())

In [ ]:
#---- TO-DO: provar que as somas das projeções dos modelos não batem no Y_hat_df

# Y_hat_df\
#     .reset_index()\
#     .assign(\
#         nivel_hierarquia = lambda x: np.where(x['unique_id'].str.count('/') == 0, 1, x['unique_id'].str.count('/') + 1)
#     )\
#     .groupby('nivel_hierarquia')[Y_hat_df.select_dtypes(include = 'number').columns]\
#     .sum()

## 4. Reconciliação

In [ ]:
reconcilers = [BottomUp(), 
               TopDown(method = 'forecast_proportions'),
               TopDown(method = 'average_proportions'),
               TopDown(method = 'proportion_averages'),
               MiddleOut(middle_level = 'region/state', top_down_method = 'forecast_proportions'),
               MiddleOut(middle_level = 'region/state', top_down_method = 'average_proportions'),
               MiddleOut(middle_level = 'region/state', top_down_method = 'proportion_averages'),
               MinTrace(method = 'ols', nonnegative = True),
               MinTrace(method = 'wls_struct', nonnegative = True),
               MinTrace(method = 'wls_var', nonnegative = True),
               MinTrace(method = 'mint_shrink', nonnegative = True),
               # MinTrace(method = 'mint_cov', nonnegative = True), # Não descomentar essa linha
               OptimalCombination(method = 'ols', nonnegative = True),
               OptimalCombination(method = 'wls_struct', nonnegative = True)
              ]

hrec = HierarchicalReconciliation(reconcilers = reconcilers)

In [ ]:
Y_rec_df = hrec.reconcile(Y_hat_df = Y_hat_df, 
                          Y_df = Y_fitted_df,
                          S = S_df,
                          tags = tags)

In [ ]:
Y_rec_df

In [ ]:
#---- A soma das projeções nos níveis de hierarquia são diferentes para os modelos sem reconciliação, exceto o Naive

# Y_rec_df\
#     .reset_index()\
#     .assign(\
#         nivel_hierarquia = lambda x: np.where(x['unique_id'].str.count('/') == 0, 1, x['unique_id'].str.count('/') + 1)
#     )\
#     .groupby('nivel_hierarquia')[Y_rec_df.select_dtypes(include = 'number').columns.tolist()]\
#     .sum()

## 5. Avaliação das métricas: WMAPE e RMSE

- Em nenhum nível a baseline (naive) foi melhor que os modelos testados

In [ ]:
def rmse(y_true, y_pred):
    
    return np.sqrt(np.mean(np.square(y_true - y_pred)))

In [ ]:
from hierarchicalforecast.evaluation import HierarchicalEvaluation

evaluator = HierarchicalEvaluation(evaluators = [rmse])

evaluation = evaluator.evaluate(
    Y_hat_df = Y_rec_df,
    Y_test_df = Y_valid_df,
    tags = tags
)

In [ ]:
#---- Separando os 2 modelos com menor RMSE, excluindo o Naive

df_metricas_modelos = pd.melt(evaluation.reset_index(), id_vars = ['level', 'metric'])\
    .sort_values(by = 'value', ascending = True)\
    .assign(\
        is_valid = lambda x: np.where((x['variable'].str.count('/') > 0) | (x['variable'] == 'Naive'), 1, 0)
    )\
    .query('is_valid == 1')\
    .groupby(['level', 'metric'])\
    .head(4)

#---- Separando os modelos Naive

df_metricas_baseline = pd.melt(evaluation.reset_index(), id_vars = ['level', 'metric'])\
    .query('variable == "Naive"')

#---- Juntando em um único DF

df_metricas = pd.concat([df_metricas_modelos, df_metricas_baseline])\
    .reset_index(drop = True)\
    .assign(\
        value = lambda x: x['value'].apply(pd.to_numeric)
    )\
    .query('level !=  "Overall"')\
    .sort_values(by = ['metric', 'level'])

df_metricas['value'] = round(df_metricas['value'], 3)
df_metricas['value_format'] = df_metricas['value'].astype(str).str.replace('.', ',')

df_metricas

In [ ]:
df_metricas.variable.unique()

In [ ]:
fig = px.bar(data_frame = df_metricas, 
             x = 'level',
             y = 'value',
             color = 'variable',
             barmode = 'group', 
             text = 'value_format', 
             template = 'plotly_white')

fig.update_layout(
    yaxis_title = 'RMSE',
    xaxis_title = 'Nível da hierarquia',
    legend_title = 'Modelo/Reconciliação',
    yaxis_visible = False,
    bargroupgap = 0.2,
    font = dict(
        size = 12
    )
)

fig.update_traces(
    textposition = 'outside'
)

fig.update_yaxes(
    visible = True,
    showticklabels = False, 
    showgrid = False
)

fig.show()

## 6. Dataframe com a tabela final

In [ ]:
def create_final_df(df_pred: pd.DataFrame, cols_split: str):

    df1 = df_pred\
        .reset_index()\
        .assign(\
            nivel_hierarquia = lambda x: np.where(x['unique_id'].str.count('/') == 0, 1, x['unique_id'].str.count('/') + 1)
        )\
        .query('nivel_hierarquia == 3')
    
    df1[cols_split] = df1['unique_id'].str.split(pat = '/', n = len(cols_split), expand = True)
    
    df1 = df1[cols_split + ['ds'] + df1.select_dtypes(include = 'number').columns.tolist()]

    return df1

In [ ]:
create_final_df(df_pred = Y_rec_df, cols_split = cols_hierarchical)

## 7. Feature importance, a partir dos modelos

**Retirado de https://mariofilho.com/como-prever-series-temporais-com-scikit-learn/**

In [ ]:
import matplotlib.pyplot as plt

for mod in list(model.models_.keys()):

    if mod == 'LinearRegression':

        pass

    else:

        plt.figure(figsize = (10, 4))

        pd.Series(model.models_[mod].feature_importances_, 
                  index = model.ts.features_order_)\
            .sort_values(ascending = False)\
            .plot\
            .bar(title = f'{mod} Feature Importance',
                 xlabel = 'Features', 
                 ylabel = 'Importance')

## 8. Visualizaçẽos das projeções pelos níveis de agregação

In [ ]:
from utilsforecast.plotting import plot_series

In [ ]:
Y_rec_df.reset_index().merge(Y_valid_df.reset_index(), on = ['unique_id', 'ds'], how = 'left')\
    .query('unique_id == "SouthCentral/Tennessee/womens_shoes"')[['ds', 'y', 'LinearRegression/MinTrace_method-ols_nonnegative-True']]

In [ ]:
px.line(data_frame = Y_rec_df.reset_index().merge(Y_valid_df.reset_index(), on = ['unique_id', 'ds'], how = 'left')\
    .query('unique_id == "SouthCentral/Tennessee/womens_shoes"')[['ds', 'y', 'LinearRegression/MinTrace_method-ols_nonnegative-True']], x = 'ds', y = ['y', 'LinearRegression/MinTrace_method-ols_nonnegative-True'])

In [ ]:
plot_series(
    Y_train_df.reset_index().query('ds >= "2008-01-01"'), 
    Y_rec_df.reset_index().merge(Y_valid_df.reset_index(), on = ['unique_id', 'ds'], how = 'left'), 
    models = ['LinearRegression/MinTrace_method-ols_nonnegative-True'],
    plot_random = False, 
)

In [ ]:
pd.melt(evaluation.reset_index(), id_vars = ['level', 'metric'])\
    .sort_values(by = 'value', ascending = True)\
    .assign(\
        is_valid = lambda x: np.where((x['variable'].str.count('/') > 0) | (x['variable'] == 'Naive'), 1, 0)
    )\
    .query('is_valid == 1')\
    .groupby(['level', 'metric'])\
    .head(4)